In [146]:
import os
import subprocess
import re
import shutil

In [147]:
class Snippet:
    def __init__(self):
        self.transcription = ''
        self.from_time = 0
        self.to_time = 0
        self.index = 0
        self.path = ''
        
    def set_from_time(self, from_time):
      self.from_time = from_time
    
    def set_to_time(self, to_time):
      self.to_time = to_time
    
    def set_transcription(self, transcription):
      self.transcription = transcription
    
    def set_index(self, index):
      self.index = index
    
    def set_path(self, path):
      self.path = path
    
    def print(self):
        print('{} - {} --> {} ---> {}'.format(self.from_time, self.to_time, self.path, self.transcription))

In [148]:
srt_file = '/home/andy/Downloads/1.srt'
video_file = '/home/andy/Downloads/MrRobot.S01E01.480p.Farda.DL.mkv'
delta = 6 # there is a 7 second delay between the subtitle and the audio file, + in +7 indicates that srt 0 = video 7
chunk_path = '/home/andy/Downloads/chunks/'
shutil.rmtree(chunk_path)
os.mkdir(chunk_path)
def string_to_time(time_string):
    split = time_string.replace(',',':').split(':')
    h = int(split[0])
    m = int(split[1])
    s = int(split[2])
    ms = int(split[3])
    return h*3600 + m*60 + s + ms/1000.0
    
def time_to_string(seconds):
    hours = seconds // (60*60)
    seconds %= (60*60)
    minutes = seconds // 60
    seconds %= 60
    ms = seconds*1000 % 1000
    return "%02i:%02i:%02i.%03i" % (hours, minutes, seconds, ms)

def extract_audio_from_video(video_path, audio_path):
    cmd = 'ffmpeg -i '+video_path+' -acodec pcm_s16le -ac 1 -ar 16000 '+audio_path
    subprocess.call(cmd, shell=True)

def read_lines(text_file_path):
    lines = []
    with open(text_file_path) as fp:
        line = fp.readline()
        lines.append(line)
        cnt = 1
        while line:
            #print("Line {}: {}".format(cnt, line.strip()))
            try:
                line = fp.readline()
                lines.append(line)
            except Exception as e:
                print(e)
            cnt += 1
    return lines

def get_snips_from_srt(srt_path):
    snips = []
    buffer_snip = Snippet()
    lines  = read_lines(srt_path)
    for index, line in enumerate(lines):
        #print('{}-->{}'.format(index, line))
        if len(line.strip())==0:
            snips.append(buffer_snip)
            buffer_snip = Snippet()
            #print('{}-->{}'.format(index, '1'))
        elif len(re.sub('^[-+]?[0-9]+$', '', line).strip()) == 0:
            buffer_snip.set_index(line)
            buffer_snip.set_path(chunk_path+'{:03d}'.format(index)+'.wav')
            #print('{}-->{}'.format(index, '2'))
        elif '-->' in line:
            from_to = line.split('-->')
            buffer_snip.set_from_time(time_to_string(string_to_time(from_to[0].strip())+delta))
            buffer_snip.set_to_time(time_to_string(string_to_time(from_to[1].strip())+delta))
            #print('{}-->{}'.format(index, '3'))
        else:
            buffer_snip.set_transcription(buffer_snip.transcription + line)
            #print('{}-->{}'.format(index, '3'))
    return snips

In [149]:
snips = get_snips_from_srt(srt_file)

'utf-8' codec can't decode byte 0xf8 in position 1114: invalid start byte


In [150]:
audio_path = '/home/andy/Downloads/1.wav'
extract_audio_from_video(video_file, audio_path)

In [151]:
for index, snip in enumerate(snips):
    snip.print()

00:00:06.000 - 00:00:07.143 --> /home/andy/Downloads/chunks/000.wav ---> That's lame.

00:00:07.144 - 00:00:09.339 --> /home/andy/Downloads/chunks/004.wav ---> Maybe I should give you a name,

00:00:09.340 - 00:00:11.011 --> /home/andy/Downloads/chunks/008.wav ---> but that's a slippery slope.

00:00:11.012 - 00:00:11.923 --> /home/andy/Downloads/chunks/012.wav ---> You're only in my head.

00:00:11.924 - 00:00:13.847 --> /home/andy/Downloads/chunks/016.wav ---> We have to remember that.

00:00:14.371 - 00:00:15.628 --> /home/andy/Downloads/chunks/020.wav ---> Shit.

00:00:15.699 - 00:00:16.509 --> /home/andy/Downloads/chunks/024.wav ---> It's actually happened,

00:00:16.511 - 00:00:18.622 --> /home/andy/Downloads/chunks/028.wav ---> I'm talking to an imaginary person.

00:00:20.299 - 00:00:21.500 --> /home/andy/Downloads/chunks/032.wav ---> What I'm about to tell you

00:00:21.500 - 00:00:22.839 --> /home/andy/Downloads/chunks/036.wav ---> is top secret,

00:00:22.999 - 00:00:25.938 

In [152]:
for index, snip in enumerate(snips):
    cmd = 'ffmpeg -i '+audio_path+' -ss '+snip.from_time+' -to '+snip.to_time+' '+snip.path
    subprocess.call(cmd, shell=True)